# Test 23: Z-test for correlated proportions

## Objective

- To investigate the significance of the difference between two correlated proportions in opinion surveys. 
- It can also be used for more general applications

## Assumptions

## Method

## Example